<a href="https://colab.research.google.com/github/Muzsek/Spam-or-Ham/blob/main/S_H_AIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir data

In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Set the path to the file you'd like to load
file_path = "spam.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uciml/sms-spam-collection-dataset",
  file_path,
  pandas_kwargs={"encoding": "latin-1"}
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

/tmp/ipython-input-1142090527.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 492k/492k [00:00<00:00, 9.64MB/s]


In [4]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
if df.columns[2] == 'Unnamed: 2':
  df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, errors='ingore') # Egyszer lefut, utána hiba
else:
    print("Deleted")

for i in df.columns:
  print(i)
df.head(5)

v1
v2


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess(text):
  text = text.lower()

  text = text.translate(str.maketrans('', '', string.punctuation))

  tokens = word_tokenize(text)

  filtered_tokens = [word for word in tokens if word not in stop_words]

  return ' '.join(filtered_tokens)

df['v2_clean'] = df['v2'].apply(preprocess)
df[['v2', 'v2_clean']].head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,v2,v2_clean
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though


In [14]:
X = df['v2_clean']
y = df['v1']

In [20]:
for i in range(5):
  print(f"{y[i]} || {X[i]}")

ham || go jurong point crazy available bugis n great world la e buffet cine got amore wat
ham || ok lar joking wif u oni
spam || free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
ham || u dun say early hor u c already say
ham || nah dont think goes usf lives around though


In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# y_encoded_list = []

# for i in range(5):
#   y_encoded_list.append(int(y_encoded[i]))

# print(list(zip(y[:5], y_encoded_list[:5]))) # Integer output

print(list(zip(y[:5], y_encoded[:5]))) # Np output

[('ham', np.int64(0)), ('ham', np.int64(0)), ('spam', np.int64(1)), ('ham', np.int64(0)), ('ham', np.int64(0))]


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [71]:
for i in range(1,5+1):
  print(f"{i}, Train label: {y_train.iloc[i]} || Train data: {X_train.iloc[i]}")

print(f"\nAll training data: {len(X_train)}")
print(f"All testing data: {len(X_test)}")

1, Train label: ham || Train data: wont wats wit guys
2, Train label: ham || Train data: ok ksry knw 2 sivatats askd
3, Train label: ham || Train data: stand away doesnt heart ache without dont wonder dont crave
4, Train label: ham || Train data: finished work yet something
5, Train label: ham || Train data: hhahhaahahah rofl wtf nig leonardo room something

All training data: 4457
All testing data: 1115


In [67]:
import pandas as pd

train_counts = pd.Series(y_train).value_counts(normalize = True)
test_counts = pd.Series(y_train).value_counts(normalize = True)

print(f"Train counts: {train_counts}")
print(f"Test counts: {test_counts}")

Train counts: v1
ham     0.865829
spam    0.134171
Name: proportion, dtype: float64
Test counts: v1
ham     0.865829
spam    0.134171
Name: proportion, dtype: float64
